In [22]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [23]:
df = pd.read_csv("random_forest_monte_carlo_trials.csv")
df.describe()

,Unnamed: 0,random_seed,num_estimators,accuracy,recall,precision,f1_score,matthews_corr_coef
count,707.000000,707.000,707.000000,707.000000,707.000000,707.000000,707.000000,707.000000
mean,353.000000,1001.000,31.428571,0.989612,0.893105,0.932013,0.911642,0.906642
std,204.237607,583.508,11.875007,0.002559,0.033876,0.025492,0.021667,0.022676
min,0.000000,1.000,15.000000,0.981641,0.784314,0.847619,0.839779,0.832751
25%,176.500000,501.000,20.000000,0.987952,0.872549,0.915442,0.896706,0.890760
50%,353.000000,1001.000,30.000000,0.989673,0.899083,0.933333,0.912442,0.907182
75%,529.500000,1501.000,45.000000,0.991394,0.917431,0.950738,0.928052,0.924166
max,706.000000,2001.000,50.000000,0.995410,0.973913,1.000000,0.958904,0.956161


In [24]:
df["num_estimators"].value_counts()/len(df)

50    0.142857
45    0.142857
35    0.142857
30    0.142857
25    0.142857
20    0.142857
15    0.142857
Name: num_estimators, dtype: float64

In [28]:
df_by_num_est = df.groupby('num_estimators').matthews_corr_coef.mean().reset_index()
df_by_num_est

,num_estimators,matthews_corr_coef
0,15,0.901357
1,20,0.903608
2,25,0.908175
3,30,0.906812
4,35,0.908093
5,45,0.908644
6,50,0.909803


### To Do List

<ul><li>Group By DWT, Num Estimator</li>
    <li>Look at maxing stats</li>
    <li>Look back at the continual 80-20 split... is training data getting too small?</li>
    <li>Decide on best DWT and Num Estimators</li>
    <li>Train with entire set, predict on Test data and submit to see how in the ball park i am</li>
    <li>Re-look at DWT thresholds, may be improvements here</li>
    <li>There is not a lot of art in implementing the Random Forest classifier, winning this competition will require insightful signal processing and novel feature extraction methods</li>
    <li>SLTM Models?</li>
    </ul>

In [25]:
a = np.array([0, 1, 3, 4, 5, 6,7 ,8 ,9,  15, 16, 17, 20])
tmp_w = 1
widths = []

for idx in range(1,len(a)):
    print(a[idx])
    if a[idx]-a[idx-1] == 1:
        tmp_w +=1
    else:
        widths.append(tmp_w)
        tmp_w = 1
widths.append(tmp_w)
widths = np.array(widths)
widths

1
3
4
5
6
7
8
9
15
16
17
20


array([2, 7, 3, 1])

In [31]:
def classification_random_forest_model(features, labels, n_value):
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators=n_value)  # Create Gaussian Classifier
    classifier.fit(features, labels.values.ravel())
    #y_predicted = classifier.predict(x_test)
    return classifier

def classification_random_forest_predict(model, x_test):
    y_predicted = model.predict(x_test)
    return y_predicted

In [33]:
# Train Model with Full Set
training_data = "/home/jeffrey/repos/VSB_Power_Line_Fault_Detection/extracted_features/train_features_"+"db4"+".csv"
df_train = pd.read_csv(training_data)
number_estimators = 50
features = df_train[["entropy", "n5", "n25", "n75", "n95", "median", "mean", "std", "var", "rms", "no_zero_crossings", "no_mean_crossings", "min_height", "max_height", "mean_height", "min_width", "max_width", "mean_width", "num_detect_peak", "num_true_peaks"]]
labels = df_train[["fault"]]
classifier = classification_random_forest_model(features, labels, number_estimators)

In [47]:
test_data = "/home/jeffrey/repos/VSB_Power_Line_Fault_Detection/extracted_features/test_features_"+"db4"+".csv"
test_meta = "/home/jeffrey/repos/VSB_Power_Line_Fault_Detection/source_data/metadata_test.csv"
df_test = pd.read_csv(test_data).drop(['Unnamed: 0'],axis=1)
df_test.head()

,signal_id,measurement_id,entropy,n5,n25,n75,n95,median,mean,std,...,no_mean_crossings,min_height,max_height,mean_height,min_width,max_width,mean_width,num_detect_peak,num_true_peaks,fault
0,8712,2904,13.592366,84936,84936,-0.047498,-0.001186,0.001160,0.048506,-0.000008,...,0.030203,-0.514307,46.612123,0.066713,1,6,1.119293,120437,107601,NaN
1,8713,2904,13.592366,84936,84936,-0.047498,-0.001186,0.001160,0.048506,-0.000008,...,0.030203,-0.514307,46.612123,0.066713,1,6,1.119293,120437,107601,NaN
2,8714,2904,13.592366,84936,84936,-0.047498,-0.001186,0.001160,0.048506,-0.000008,...,0.030203,-0.514307,46.612123,0.066713,1,6,1.119293,120437,107601,NaN
3,8715,2905,13.592366,1030,1030,-0.000386,-0.000155,0.000147,0.000360,0.000010,...,0.000233,0.000413,0.168632,0.001385,1,2,1.084187,4340,4003,NaN
4,8716,2905,13.592366,1030,1030,-0.000386,-0.000155,0.000147,0.000360,0.000010,...,0.000233,0.000413,0.168632,0.001385,1,2,1.084187,4340,4003,NaN


In [48]:
# Make Predictions
test_data = "/home/jeffrey/repos/VSB_Power_Line_Fault_Detection/extracted_features/test_features_"+"db4"+".csv"
test_meta = "/home/jeffrey/repos/VSB_Power_Line_Fault_Detection/source_data/metadata_test.csv"
df_test = pd.read_csv(test_data)
test_features = df_test[["entropy", "n5", "n25", "n75", "n95", "median", "mean", "std", "var", "rms", "no_zero_crossings", "no_mean_crossings", "min_height", "max_height", "mean_height", "min_width", "max_width", "mean_width", "num_detect_peak", "num_true_peaks"]]
predicted_faults = classification_random_forest_predict(classifier, test_features)

In [49]:
predicted_faults

array([0, 0, 0, ..., 0, 0, 0])

In [50]:
print(len(predicted_faults))
print(len(df_test))

20337
20337


In [51]:
df_test["fault"] = predicted_faults

In [53]:
df_test.head()

,Unnamed: 0,signal_id,measurement_id,entropy,n5,n25,n75,n95,median,mean,...,no_mean_crossings,min_height,max_height,mean_height,min_width,max_width,mean_width,num_detect_peak,num_true_peaks,fault
0,0,8712,2904,13.592366,84936,84936,-0.047498,-0.001186,0.001160,0.048506,...,0.030203,-0.514307,46.612123,0.066713,1,6,1.119293,120437,107601,0
1,1,8713,2904,13.592366,84936,84936,-0.047498,-0.001186,0.001160,0.048506,...,0.030203,-0.514307,46.612123,0.066713,1,6,1.119293,120437,107601,0
2,2,8714,2904,13.592366,84936,84936,-0.047498,-0.001186,0.001160,0.048506,...,0.030203,-0.514307,46.612123,0.066713,1,6,1.119293,120437,107601,0
3,3,8715,2905,13.592366,1030,1030,-0.000386,-0.000155,0.000147,0.000360,...,0.000233,0.000413,0.168632,0.001385,1,2,1.084187,4340,4003,0
4,4,8716,2905,13.592366,1030,1030,-0.000386,-0.000155,0.000147,0.000360,...,0.000233,0.000413,0.168632,0.001385,1,2,1.084187,4340,4003,0


In [54]:
sum(predicted_faults)

411

In [55]:
df_test["fault"].sum()

411

In [56]:
df_test = df_test.drop(['Unnamed: 0'],axis=1)

In [57]:
df_test[["signal_id", "fault"]].to_csv("prediction_submission_"+"db4"+".csv", sep=",", index="False")

In [58]:
df_test["signal_id"][0]

8712

In [59]:
df_test.describe()

,signal_id,measurement_id,entropy,n5,n25,n75,n95,median,mean,std,...,no_mean_crossings,min_height,max_height,mean_height,min_width,max_width,mean_width,num_detect_peak,num_true_peaks,fault
count,20337.000000,20337.00000,20337.000000,20337.000000,20337.000000,2.033700e+04,2.033700e+04,2.033700e+04,2.033700e+04,2.033700e+04,...,2.033700e+04,20337.000000,20337.000000,20337.000000,20337.000000,20337.000000,20337.000000,20337.000000,20337.000000,20337.000000
mean,18880.000000,6293.00000,13.592365,49152.003245,49152.009441,-1.148637e-01,-1.734675e-02,1.733564e-02,1.145368e-01,-5.362189e-07,...,9.165688e-02,1.314677,83.440368,5.389474,1.007228,5.068152,1.214473,62565.263756,56150.547868,0.020209
std,5870.930548,1956.97683,0.000004,36698.782935,36698.782963,2.769840e-01,2.793258e-02,2.795163e-02,2.755939e-01,6.765953e-04,...,1.972283e-01,4.665560,92.769728,11.231866,0.086437,3.294829,0.229898,52324.498221,47019.390229,0.140720
min,8712.000000,2904.00000,13.592080,6.000000,6.000000,-8.098634e+00,-2.498850e-01,1.144687e-07,6.736626e-07,-1.142379e-02,...,4.808215e-07,-18.772525,0.000018,-0.000005,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,13796.000000,4598.00000,13.592366,11056.000000,11056.000000,-1.875538e-01,-2.231573e-02,2.317519e-04,5.038802e-03,-1.094377e-05,...,7.036908e-03,-0.452645,7.940841,0.033087,1.000000,3.000000,1.099601,975.000000,710.000000,0.000000
50%,18880.000000,6293.00000,13.592366,54062.000000,54064.000000,-4.765637e-02,-4.817747e-03,4.824714e-03,4.783226e-02,4.775493e-07,...,3.688348e-02,-0.000487,41.581537,0.139956,1.000000,4.000000,1.113634,96016.000000,86707.000000,0.000000
75%,23964.000000,7988.00000,13.592366,77119.000000,77115.000000,-5.069583e-03,-2.280028e-04,2.221928e-02,1.870626e-01,1.300917e-05,...,9.038329e-02,0.819182,140.348864,5.355763,1.000000,6.000000,1.242172,110131.000000,98540.000000,0.000000
max,29048.000000,9682.00000,13.592366,248512.000000,248510.000000,-9.186898e-07,-8.198267e-08,2.429994e-01,8.107124e+00,1.061686e-02,...,3.080084e+00,70.115919,256.173007,127.806427,3.000000,26.000000,3.000000,163441.000000,152496.000000,1.000000
